In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "name": {"type": "string"},
        "height": {"type": "integer"},
        "hair_color": {"type": "string"},
    },
    "required": ["name", "height"],
}

# Input 
inp = """Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde."""

# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)
chain.run(inp)

[{'name': 'Alex', 'height': 5, 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': 6, 'hair_color': 'brunette'}]

# Testing convertion between feet and centimeters

In [3]:
inp = """Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde. Lucas is 50 centimeters taller than Alex"""
chain.run(inp)

[{'name': 'Alex', 'height': 5, 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': 6, 'hair_color': 'brunette'},
 {'name': 'Lucas', 'height': 55}]

# Upgrading model to gpt-4

In [4]:
# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-4")
chain = create_extraction_chain(schema, llm)
chain.run(inp)

[{'name': 'Alex', 'height': 5, 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': 6, 'hair_color': 'brunette'},
 {'name': 'Lucas', 'height': 5.64}]

In [5]:
def feet_to_cm(x):
    '''Conversion from feet to cm, is cm = feet/3.281'''
    return x / 3.281

#not good enough =/
feet_to_cm(5.64) - feet_to_cm(5)


0.19506248095092937

In [6]:
feet_to_cm(5.64)

1.7189881133800669

# Helping the LLM with some context

In [12]:
chain.run(input=inp, context='Conversion from feet to cm, is cm = feet/3.281')
#still not good

[{'name': 'Alex', 'height': 5, 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': 6, 'hair_color': 'brunette'},
 {'name': 'Lucas', 'height': 5.5}]

In [14]:
chain.run(input=inp, context='Remember that feet and centimeters are different measures. Conversion from cm to feet, is feet = cm*3.281')

[{'name': 'Alex', 'height': 5, 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': 6, 'hair_color': 'brunette'},
 {'name': 'Lucas', 'height': 6.64}]

In [15]:
#nice!
feet_to_cm(6.64) - feet_to_cm(5)

0.49984760743675705

#  What if we go back to gpt-3.5 and provide the context?

In [24]:
from langchain.prompts import ChatPromptTemplate

In [28]:
# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
prompt = '''Extract and save the relevant entities mentioned\
in the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

Passage:'''

inp = """Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde. Lucas is 50 centimeters taller than Alex"""

chain = create_extraction_chain(schema, llm, verbose=True, prompt=ChatPromptTemplate.from_template(prompt))
chain.run(input=inp, context='Remember that feet and centimeters are different measures. Conversion from cm to feet, is feet = cm*3.281')



> Entering new LLMChain chain...
Prompt after formatting:
Human: Extract and save the relevant entities mentionedin the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

Passage:

> Finished chain.


[{'name': 'John', 'height': 180, 'hair_color': 'brown'},
 {'name': 'Sarah', 'height': 165, 'hair_color': 'blonde'}]

In [ ]:
# who are john and sarah?!?!